In [1]:
import os, openai, requests, json, zeep
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv, find_dotenv
from requests.auth import HTTPBasicAuth
from zeep import Client 
from zeep.wsse.username import UsernameToken
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from schema.Staffing import *

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.getenv('OPENAI_API_KEY')

TENANT = 'wdmarketdesk_dpt1'
WD_USER_ID = os.getenv('WD_USER_ID')
WD_PWD = os.getenv('WD_PWD')
WD_Absence_URL = "https://impl-services1.wd12.myworkday.com/ccx/service/customreport2/wdmarketdesk_dpt1/xjin-impl/Worker_Absence_Data?format=json"
WD_COMP_URL = "https://impl-services1.wd12.myworkday.com/ccx/service/customreport2/wdmarketdesk_dpt1/xjin-impl/Worker_Comp_Data?format=json"
HR_URL = "https://impl-services1.wd12.myworkday.com/ccx/service/wdmarketdesk_dpt1/Human_Resources/v42.0?wsdl"
STAFFING_URL = "https://impl-services1.wd12.myworkday.com/ccx/service/wdmarketdesk_dpt1/Staffing/v41.1?wsdl"

basicAuth = HTTPBasicAuth(WD_USER_ID, WD_PWD)

#HR_Client = Client(HR_URL, wsse=UsernameToken(WD_USER_ID + '@' + TENANT, WD_PWD)) 
#Staffing_Client = Client(STAFFING_URL, wsse=UsernameToken(WD_USER_ID + '@' + TENANT, WD_PWD))

model_name = "gpt-4-1106-preview"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)


c:\Users\sures\anaconda3\envs\lc-agent-2\lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\sures\anaconda3\envs\lc-agent-2\lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [19]:
#print(BusinessProcessParametersType.schema_json(indent=2))

parser = PydanticOutputParser(pydantic_object=BusinessProcessParametersType)

input_query = "Generate output for business process parameters using defaults as True. Consider 'testing' for comments"

prompt = PromptTemplate(
    template="Answer the user query. Note that Comment_Data, Worker_Reference, ID type is Employee_ID. \
        Please do not output any text before or after JSON structure \
        Do not assume any values for any fields \
        \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions() + 
        "Do not prefix or suffix the final output with special characters or JSON keys"},
)

_input = prompt.format_prompt(query=input_query)

#print(parser.get_format_instructions())
output = model(_input.to_string())


In [21]:

#parser.parse(output)

#print(output.strip("`").strip('json'))

print(output.replace("Value", "_value_1"))

#print(parser.get_format_instructions())

{
  "Auto_Complete": true,
  "Run_Now": true,
  "Discard_On_Exit_Validation_Error": true,
  "Comment_Data": {
    "Comment": "testing",
    "Worker_Reference": {
      "ID": [
        {
          "_value_1": "Employee_ID",
          "type": "Employee_ID"
        }
      ],
      "Descriptor": "Employee_ID"
    }
  },
  "Business_Process_Attachment_Data": []
}


In [4]:
parser = PydanticOutputParser(pydantic_object=AddAdditionalJobDataType)

input_query = "Generate output for Additional Job. Requisition is 'Test-Req'"

prompt = PromptTemplate(
    template="Answer the user query. Note that Employee_Reference ID type is Employee_ID. \
        Organization_Reference ID type is Organization_Reference_ID \
        \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=input_query)

#print(parser.get_format_instructions())
output = model(_input.to_string())


In [5]:

#parser.parse(output)

print(output.strip("`").strip('json'))

#print(output)

#print(parser.get_format_instructions())

Based on the provided schema and the information that the requisition is 'Test-Req', here is a JSON instance that conforms to the schema:

```json
{
  "Employee_Reference": {
    "ID": [
      {
        "Value": "12345",
        "type": "Employee_ID"
      }
    ],
    "Descriptor": "Employee for Additional Job"
  },
  "Organization_Reference": {
    "ID": [
      {
        "Value": "67890",
        "type": "Organization_Reference_ID"
      }
    ],
    "Descriptor": "Organization for Additional Job"
  },
  "Add_Additional_Job_Event_Data": {
    "Position_ID": "Position-123",
    "Position_Details": {
      "Job_Profile_Reference": {
        "ID": [
          {
            "Value": "JobProfile-123",
            "type": "Job_Profile_ID"
          }
        ],
        "Descriptor": "Job Profile for Additional Job"
      },
      "Location_Reference": {
        "ID": [
          {
            "Value": "Location-123",
            "type": "Location_ID"
          }
        ],
        "Descri

In [32]:
from langchain.output_parsers import PydanticOutputParser, CommaSeparatedListOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

class PlanItem(BaseModel):
    step: str
    tools: Optional[str] = []
    data_sources: Optional[str] = []
    sub_steps_needed: str

class Plan(BaseModel):
    plan: List[PlanItem]


parser = PydanticOutputParser(pydantic_object=Plan)
parser_2 = CommaSeparatedListOutputParser()

input_query = "Generate details for plan item 1"
input_query_2 = ""

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_2 = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser_2.get_format_instructions()},

)

_input = prompt.format_prompt(query=input_query)

#print(parser.get_format_instructions())
output = model(_input.to_string())

parser.parse(output)

Plan(plan=[PlanItem(step='Step 1 description', tools='List of tools used in Step 1', data_sources='List of data sources for Step 1', sub_steps_needed='Description of sub-steps needed for Step 1')])

## Add Additional Job